<h1>Anotador de Podócitos (Versão Python)</h1>

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 10 02:24:18 2020

@author: george
"""
#Garantindo a importação e instalação do pacote tkinter 
try:
    import tkinter as tk
    from tkinter import ttk, filedialog, Label, Button, Menu
    from tkinter.messagebox import showinfo
except ImportError:
    from pip._internal import main as pip
    pip(['install', '--user', 'python-tk'])
    import tkinter as tk
    from tkinter import ttk, filedialog, Label, Button, Menu
    from tkinter.messagebox import showinfo

#Garantindo a importação e instalação do pacote PIL
try:
    from PIL import ImageTk, Image
    from PIL import _tkinter_finder
except ImportError:
    from pip._internal import main as pip
    pip(['install', '--user', 'pil'])
    from PIL import ImageTk, Image
    from PIL import _tkinter_finder ##Sem essa importação dá erro no executável


#import tkFileDialog

filename = 0
array_anotacao=[]#armazenar aqui coordenadas iniciais e finais de cada retângulo
only_name=""
array_save_rets = []
class App(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self._createVariables(parent)
        self._createCanvas()
        self._createCanvasBinding()
            
    def _createVariables(self, parent):
        self.parent = parent
        self.rectx0 = 0
        self.recty0 = 0
        self.rectx1 = 0
        self.recty1 = 0
        self.rectid = None
        self.move = False
        
    def _createCanvas(self):
        canvas_w= 800
        canvas_h = 600
        global filename
        self.canvas = tk.Canvas(self.parent, width = canvas_w, height = canvas_h)
        self.canvas.pack(expand = "yes", fill = "both")#self.canvas.grid(row=0, column=0, sticky="nsew")
        self.im = ImageTk.PhotoImage(file = filename)
        self.im.width = canvas_w
        self.im.height = canvas_h
        self.canvas.create_image(0, 0, image = self.im, anchor = "nw")
        #self.canvas.create_image(10, 10, image = im, anchor = 'nw')
        self.canvas.focus_set()
  
    def _createCanvasBinding(self):
        self.canvas.bind( "<Button-1>", self.startRect )
        self.canvas.bind( "<ButtonRelease-1>", self.stopRect )
        self.canvas.bind( "<Motion>", self.movingRect )
        self.canvas.bind("<KeyPress>",self.atalhos)
    
    def atalhos(self, event):
        #print event.char
        if (event.char=="s"): 
            geraTxt()
        if (event.char=="d"): 
            self.deleteRet()
        if (event.char=="h"): 
            popup_ajuda()
        if (event.char==""): 
            array_anotacao.clear()
            self.parent.destroy()
            
    
    #Método para excluir os retângulos
    def deleteRet(self):
        #showinfo("Window", "Testando")
        #deletando os rets pelo ordm de que foram armazenados no array 
        global array_save_rets
        self.size = len(array_save_rets)
        self.canvas.delete(array_save_rets[self.size-1]) #excluindo o retangulo
        
        #eliminando o registro do retangulo no array
        del(array_save_rets[self.size-1])
        #self.canvas.delete(self.rect)
        
        #preciso também excluir as coordenadas do retangulo que foram armaznadas no array de coordenadas, que será registrado no txt
        global array_anotacao
        self.size_array_anotacao = len(array_anotacao)
        del(array_anotacao[self.size_array_anotacao-1])
        del(array_anotacao[self.size_array_anotacao-2])
        print ("Retangulo excluído")
    
    def startRect(self, event):
        self.move = True
        
        #Translate mouse screen x0,y0 coordinates to canvas coordinates
        self.rectx0 = self.canvas.canvasx(event.x)
        self.recty0 = self.canvas.canvasy(event.y)
        
        #Create rectangle
        self.rect = self.canvas.create_rectangle(
        self.rectx0, self.recty0, self.rectx0, self.recty0, outline='yellow', width=2) #fill="#ffffff"
        
        #save os rects em um array para depois excluí-los na ordem de criação
        global array_save_rets
        array_save_rets = array_save_rets + [self.rect]
        
        #Get rectangle"s canvas object ID
        self.rectid = self.canvas.find_closest(self.rectx0, self.recty0, halo=2)
        print("Rectangle {0} started at {1} {2} {3} {4} ".
        format(self.rect, self.rectx0, self.recty0, self.rectx0,
        self.recty0))
        #global array_temp
        #print self.rect
        global array_anotacao
        array_anotacao=array_anotacao+[[self.rectx0,self.recty0]]
    
    def movingRect(self, event):
        if self.move:
            #Translate mouse screen x1,y1 coordinates to canvas coordinates
            self.rectx1 = self.canvas.canvasx(event.x)
            self.recty1 = self.canvas.canvasy(event.y)
            #Modify rectangle x1, y1 coordinates
            self.canvas.coords(self.rectid, self.rectx0, self.recty0,
            self.rectx1, self.recty1)
            #print("Rectangle x1, y1 = ", self.rectx1, self.recty1)
        
    def stopRect(self, event):
        self.move = False
        #Translate mouse screen x1,y1 coordinates to canvas coordinates
        self.rectx1 = self.canvas.canvasx(event.x)
        self.recty1 = self.canvas.canvasy(event.y)
        #Modify rectangle x1, y1 coordinates (final)
        self.canvas.coords(self.rectid, self.rectx0, self.recty0,
        self.rectx1, self.recty1)
        global array_anotacao
        array_anotacao=array_anotacao+[[self.rectx1,self.recty1]]
        print("Rectangle x1, y1 = ", self.rectx1, self.recty1)
        print("Rectangle ended")
        
def UploadAction():
    global filename
    filename= filedialog.askopenfilename()
    print('Selected:', filename)
    
    #redimensionando a imagem e salvando em uma nova dimensão
    #basewidth = 300
    img = Image.open(filename)
    #wpercent = (basewidth/float(img.size[0]))
    #hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((500,500), Image.ANTIALIAS)
    img.save(filename) 

    nome_da_foto = open(filename) #peguei aqui novamente pra conseguir depois acessar apenas o nome e gerar um txt com mesmo título
    # Explode o arquivo em cada ponto (.)
    explod_file = nome_da_foto.name.split('.')
    # Apenas o nome do arquivo
    global only_name
    only_name = '.'.join(explod_file[:-1])
    print(only_name)

    root2 = tk.Toplevel()
    #root = tk.Tk()
    menubar = Menu(root2, background='white', foreground='black', activebackground='#004c99', activeforeground='white')
    root2.config(menu=menubar)
    root2.geometry( "500x500" )
    root2.title("Anotador de Podócitos")
    app = App(root2)
    
    menubar.add_command(label='Desfazer (d)', command=app.deleteRet)
    menubar.add_separator()
    menubar.add_command(label='Salvar (s)', command=geraTxt)
    menubar.add_separator()
    menubar.add_command(label='Ajuda (h)', command=popup_ajuda)
    menubar.add_separator()
    menubar.add_command(label='Sair (Esc)', command=root2.destroy)      
    root2.mainloop()

def popup_ajuda():
    win = tk.Toplevel()
    win.wm_title("Instruções de Utilização")
    
    win.geometry("600x280")
    l1 = tk.Label(win, wraplength=580, text="(1) Clique uma vez nas mediações do podócito para criar o ponto inicial, clique uma segundo vez em um segundo ponto. Um retângulo será formado. (2) Após demarcar as regiões de todos os podócitos, clique em salvar. (3) Neste momento a marcação já foi salva, agora você pode fechar a tela ou cicar na opção sair. (4) Escolha outra imagem e repita o processo.", font=('Verdana','12','bold'))
    l1.grid(row=0, column=0)
    
    l2 = tk.Label(win, pady=15, wraplength=580, text="Caso você tenha marcado um podócito errado, poderá desfazer a marcação clicando no botão _Desfazer_", font=('Arial','12', 'bold'))
    l2.grid(row=1, column=0)
     
    b = ttk.Button(win, text="Okay", command=win.destroy)
    b.grid(row=5, column=0)

def geraTxt():
    global array_anotacao
    array_anotacao
    array_temp = str(array_anotacao)
    global only_name
    only_name = only_name+".txt"
    arquivo = open(only_name, 'w')
    arquivo.write(array_temp)
    arquivo.close()
    array_anotacao = []
    showinfo("Obrigado!", "Anotação salva com sucesso!")
    
class Application(ttk.Frame):

    def __init__(self, master):
        ttk.Frame.__init__(self, master)
        self.pack()
        
        self.titulo=Label(self, text = 'Bem vindo! Por favor, clique no botão para escolher uma imagem ', font=('Verdana','12','bold'), padx=5, pady=15)
        self.titulo.pack()        
        
        self.b01=Button(self,text='Escolher imagem', font=('Verdana','12',), command=UploadAction)
        self.b01['padx'],self.b01['pady'] = 10, 5
        self.b01['bg']='gray'
        self.b01.pack()

root = tk.Tk()
root.title("Pathospotter - Anotação de Imagens")
root.geometry("800x100")
app = Application(root)

root.mainloop()

('Selected:', '/home/george/anotacao natanael/original/01.jpg')
/home/george/anotacao natanael/original/01
Rectangle 2 started at 284.0 45.0 284.0 45.0 
('Rectangle x1, y1 = ', 305.0, 64.0)
Rectangle ended
Rectangle 3 started at 440.0 156.0 440.0 156.0 
('Rectangle x1, y1 = ', 419.0, 127.0)
Rectangle ended
Rectangle 4 started at 338.0 292.0 338.0 292.0 
('Rectangle x1, y1 = ', 358.0, 270.0)
Rectangle ended
Retangulo excluído
Rectangle 5 started at 338.0 267.0 338.0 267.0 
('Rectangle x1, y1 = ', 362.0, 297.0)
Rectangle ended
Rectangle 6 started at 253.0 268.0 253.0 268.0 
('Rectangle x1, y1 = ', 276.0, 295.0)
Rectangle ended
Rectangle 7 started at 201.0 295.0 201.0 295.0 
('Rectangle x1, y1 = ', 221.0, 268.0)
Rectangle ended
Retangulo excluído
Rectangle 8 started at 203.0 299.0 203.0 299.0 
('Rectangle x1, y1 = ', 222.0, 266.0)
Rectangle ended
Rectangle 9 started at 118.0 228.0 118.0 228.0 
('Rectangle x1, y1 = ', 95.0, 255.0)
Rectangle ended
Rectangle 10 started at 43.0 220.0 43.0 220

('Rectangle x1, y1 = ', 279.0, 443.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/07.jpg')
/home/george/anotacao natanael/original/07
Rectangle 2 started at 176.0 86.0 176.0 86.0 
('Rectangle x1, y1 = ', 161.0, 109.0)
Rectangle ended
Rectangle 3 started at 164.0 213.0 164.0 213.0 
('Rectangle x1, y1 = ', 183.0, 233.0)
Rectangle ended
Rectangle 4 started at 123.0 219.0 123.0 219.0 
('Rectangle x1, y1 = ', 139.0, 239.0)
Rectangle ended
Rectangle 5 started at 319.0 378.0 319.0 378.0 
('Rectangle x1, y1 = ', 304.0, 401.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/08.jpg')
/home/george/anotacao natanael/original/08
Rectangle 2 started at 218.0 177.0 218.0 177.0 
('Rectangle x1, y1 = ', 242.0, 152.0)
Rectangle ended
Rectangle 3 started at 381.0 176.0 381.0 176.0 
('Rectangle x1, y1 = ', 400.0, 196.0)
Rectangle ended
Rectangle 4 started at 394.0 330.0 394.0 330.0 
('Rectangle x1, y1 = ', 408.0, 315.0)
Rectangle ended
Retangulo excluído
R

Rectangle 7 started at 244.0 424.0 244.0 424.0 
('Rectangle x1, y1 = ', 224.0, 445.0)
Rectangle ended
Rectangle 8 started at 304.0 471.0 304.0 471.0 
('Rectangle x1, y1 = ', 325.0, 487.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/13.jpg')
/home/george/anotacao natanael/original/13
Rectangle 2 started at 362.0 97.0 362.0 97.0 
('Rectangle x1, y1 = ', 383.0, 122.0)
Rectangle ended
Rectangle 3 started at 365.0 257.0 365.0 257.0 
('Rectangle x1, y1 = ', 380.0, 274.0)
Rectangle ended
Rectangle 4 started at 285.0 447.0 285.0 447.0 
('Rectangle x1, y1 = ', 266.0, 470.0)
Rectangle ended
Rectangle 5 started at 142.0 389.0 142.0 389.0 
('Rectangle x1, y1 = ', 160.0, 409.0)
Rectangle ended
Rectangle 6 started at 33.0 291.0 33.0 291.0 
('Rectangle x1, y1 = ', 50.0, 315.0)
Rectangle ended
Rectangle 7 started at 53.0 186.0 53.0 186.0 
('Rectangle x1, y1 = ', 69.0, 211.0)
Rectangle ended
Retangulo excluído
Retangulo excluído
Retangulo excluído
Rectangle 8 started at 142.0

('Rectangle x1, y1 = ', 149.0, 405.0)
Rectangle ended
Rectangle 13 started at 59.0 219.0 59.0 219.0 
('Rectangle x1, y1 = ', 37.0, 240.0)
Rectangle ended
Rectangle 14 started at 54.0 177.0 54.0 177.0 
('Rectangle x1, y1 = ', 77.0, 199.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/22.jpg')
/home/george/anotacao natanael/original/22
Rectangle 2 started at 279.0 78.0 279.0 78.0 
('Rectangle x1, y1 = ', 255.0, 103.0)
Rectangle ended
Rectangle 3 started at 435.0 191.0 435.0 191.0 
('Rectangle x1, y1 = ', 462.0, 222.0)
Rectangle ended
Rectangle 4 started at 251.0 118.0 251.0 118.0 
('Rectangle x1, y1 = ', 232.0, 152.0)
Rectangle ended
Retangulo excluído
Rectangle 5 started at 231.0 123.0 231.0 123.0 
('Rectangle x1, y1 = ', 255.0, 151.0)
Rectangle ended
Rectangle 6 started at 64.0 331.0 64.0 331.0 
('Rectangle x1, y1 = ', 43.0, 365.0)
Rectangle ended
Rectangle 7 started at 398.0 379.0 398.0 379.0 
('Rectangle x1, y1 = ', 374.0, 403.0)
Rectangle ended
('Selected:',

Rectangle 3 started at 294.0 142.0 294.0 142.0 
('Rectangle x1, y1 = ', 326.0, 168.0)
Rectangle ended
Rectangle 4 started at 325.0 186.0 325.0 186.0 
('Rectangle x1, y1 = ', 355.0, 218.0)
Rectangle ended
Rectangle 5 started at 413.0 196.0 413.0 196.0 
('Rectangle x1, y1 = ', 436.0, 225.0)
Rectangle ended
Rectangle 6 started at 415.0 241.0 415.0 241.0 
('Rectangle x1, y1 = ', 442.0, 264.0)
Rectangle ended
Rectangle 7 started at 405.0 312.0 405.0 312.0 
('Rectangle x1, y1 = ', 437.0, 338.0)
Rectangle ended
Rectangle 8 started at 428.0 384.0 428.0 384.0 
('Rectangle x1, y1 = ', 460.0, 419.0)
Rectangle ended
Rectangle 9 started at 101.0 313.0 101.0 313.0 
('Rectangle x1, y1 = ', 130.0, 342.0)
Rectangle ended
Rectangle 10 started at 97.0 271.0 97.0 271.0 
('Rectangle x1, y1 = ', 70.0, 308.0)
Rectangle ended
Rectangle 11 started at 97.0 147.0 97.0 147.0 
('Rectangle x1, y1 = ', 121.0, 172.0)
Rectangle ended
Rectangle 12 started at 161.0 105.0 161.0 105.0 
('Rectangle x1, y1 = ', 134.0, 130.0

...

Rectangle 16 started at 213.0 235.0 213.0 235.0 
('Rectangle x1, y1 = ', 184.0, 267.0)
Rectangle ended
Rectangle 17 started at 230.0 374.0 230.0 374.0 
('Rectangle x1, y1 = ', 257.0, 403.0)
Rectangle ended
Rectangle 18 started at 233.0 369.0 233.0 369.0 
('Rectangle x1, y1 = ', 214.0, 408.0)
Rectangle ended
Rectangle 19 started at 284.0 428.0 284.0 428.0 
('Rectangle x1, y1 = ', 311.0, 458.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/72.jpg')
/home/george/anotacao natanael/original/72
Rectangle 2 started at 250.0 55.0 250.0 55.0 
('Rectangle x1, y1 = ', 237.0, 76.0)
Rectangle ended
Rectangle 3 started at 189.0 57.0 189.0 57.0 
('Rectangle x1, y1 = ', 175.0, 81.0)
Rectangle ended
Rectangle 4 started at 165.0 75.0 165.0 75.0 
('Rectangle x1, y1 = ', 182.0, 95.0)
Rectangle ended
Rectangle 5 started at 143.0 126.0 143.0 126.0 
('Rectangle x1, y1 = ', 162.0, 151.0)
Rectangle ended
Rectangle 6 started at 233.0 199.0 233.0 199.0 
('Rectangle x1, y1 = ', 214.0, 217

('Rectangle x1, y1 = ', 244.0, 328.0)
Rectangle ended
Rectangle 14 started at 279.0 410.0 279.0 410.0 
('Rectangle x1, y1 = ', 261.0, 428.0)
Rectangle ended
Rectangle 15 started at 243.0 418.0 243.0 418.0 
('Rectangle x1, y1 = ', 226.0, 433.0)
Rectangle ended
Retangulo excluído
Rectangle 16 started at 245.0 417.0 245.0 417.0 
('Rectangle x1, y1 = ', 228.0, 431.0)
Rectangle ended
Rectangle 17 started at 137.0 416.0 137.0 416.0 
('Rectangle x1, y1 = ', 155.0, 437.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/79.jpg')
/home/george/anotacao natanael/original/79
Rectangle 2 started at 247.0 215.0 247.0 215.0 
('Rectangle x1, y1 = ', 265.0, 234.0)
Rectangle ended
Rectangle 3 started at 279.0 290.0 279.0 290.0 
('Rectangle x1, y1 = ', 299.0, 313.0)
Rectangle ended
Rectangle 4 started at 263.0 370.0 263.0 370.0 
('Rectangle x1, y1 = ', 241.0, 401.0)
Rectangle ended
Retangulo excluído
Rectangle 5 started at 248.0 373.0 248.0 373.0 
('Rectangle x1, y1 = ', 265.0, 391.

('Rectangle x1, y1 = ', 269.0, 370.0)
Rectangle ended
Rectangle 8 started at 287.0 338.0 287.0 338.0 
('Rectangle x1, y1 = ', 316.0, 363.0)
Rectangle ended
Rectangle 9 started at 311.0 352.0 311.0 352.0 
('Rectangle x1, y1 = ', 339.0, 378.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/83b.jpg')
/home/george/anotacao natanael/original/83b
Rectangle 2 started at 137.0 68.0 137.0 68.0 
('Rectangle x1, y1 = ', 167.0, 100.0)
Rectangle ended
Rectangle 3 started at 104.0 210.0 104.0 210.0 
('Rectangle x1, y1 = ', 76.0, 260.0)
Rectangle ended
Rectangle 4 started at 213.0 426.0 213.0 426.0 
('Rectangle x1, y1 = ', 262.0, 462.0)
Rectangle ended
Rectangle 5 started at 305.0 380.0 305.0 380.0 
('Rectangle x1, y1 = ', 338.0, 414.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/84.jpg')
/home/george/anotacao natanael/original/84
Rectangle 2 started at 99.0 117.0 99.0 117.0 
('Rectangle x1, y1 = ', 85.0, 142.0)
Rectangle ended
Rectangle 3 started at

Rectangle 5 started at 230.0 36.0 230.0 36.0 
('Rectangle x1, y1 = ', 245.0, 59.0)
Rectangle ended
Rectangle 6 started at 267.0 159.0 267.0 159.0 
('Rectangle x1, y1 = ', 286.0, 179.0)
Rectangle ended
Rectangle 7 started at 280.0 177.0 280.0 177.0 
('Rectangle x1, y1 = ', 300.0, 196.0)
Rectangle ended
Rectangle 8 started at 195.0 357.0 195.0 357.0 
('Rectangle x1, y1 = ', 215.0, 379.0)
Rectangle ended
Rectangle 9 started at 228.0 379.0 228.0 379.0 
('Rectangle x1, y1 = ', 250.0, 403.0)
Rectangle ended
Rectangle 10 started at 44.0 226.0 44.0 226.0 
('Rectangle x1, y1 = ', 69.0, 258.0)
Rectangle ended
Rectangle 11 started at 51.0 176.0 51.0 176.0 
('Rectangle x1, y1 = ', 72.0, 211.0)
Rectangle ended
('Selected:', '/home/george/anotacao natanael/original/92.jpg')
/home/george/anotacao natanael/original/92
Rectangle 2 started at 178.0 83.0 178.0 83.0 
('Rectangle x1, y1 = ', 200.0, 106.0)
Rectangle ended
Rectangle 3 started at 230.0 78.0 230.0 78.0 
('Rectangle x1, y1 = ', 245.0, 98.0)
Rec

('Rectangle x1, y1 = ', 295.0, 57.0)
Rectangle ended
Rectangle 3 started at 323.0 30.0 323.0 30.0 
('Rectangle x1, y1 = ', 341.0, 52.0)
Rectangle ended
Rectangle 4 started at 305.0 51.0 305.0 51.0 
('Rectangle x1, y1 = ', 320.0, 73.0)
Rectangle ended
Rectangle 5 started at 421.0 259.0 421.0 259.0 
('Rectangle x1, y1 = ', 401.0, 285.0)
Rectangle ended
Rectangle 6 started at 407.0 328.0 407.0 328.0 
('Rectangle x1, y1 = ', 395.0, 356.0)
Rectangle ended
Retangulo excluído
Rectangle 7 started at 398.0 330.0 398.0 330.0 
('Rectangle x1, y1 = ', 416.0, 365.0)
Rectangle ended
Retangulo excluído
Rectangle 8 started at 413.0 333.0 413.0 333.0 
('Rectangle x1, y1 = ', 395.0, 358.0)
Rectangle ended
Rectangle 9 started at 283.0 441.0 283.0 441.0 
('Rectangle x1, y1 = ', 254.0, 462.0)
Rectangle ended
Rectangle 10 started at 229.0 454.0 229.0 454.0 
('Rectangle x1, y1 = ', 202.0, 476.0)
Rectangle ended
Rectangle 11 started at 156.0 415.0 156.0 415.0 
('Rectangle x1, y1 = ', 179.0, 441.0)
Rectangle e